# NLP Final Project: Topic Modeling

# Matthew Przybyla

# Table of Contents

Movie Review Data

Hypothesis

Results A

Summary A

Results B

Summary B

Conclusion

# Motivation

To get good hands-on experience working with NMF and/or LDA and the necessary Python libraries to analyze and visualize results. 

# Business Understanding

What is topic-modelling?

It is a statistical model for discovering the abstract "topics" that occur in a collection of documents. Topic modeling is a frequently used text-mining tool for discovery of hidden semantic structures in a text body. 

The "topics" produced by topic modeling techniques are clusters of similar words. A topic model captures this intuition in a mathematical framework, which allows examining a set of documents and discovering, based on the statistics of the words in each, what the topics might be and what each document's balance of topics is. It involves various techniques of dimensionality reduction(mostly non-linear) and unsupervised learning like LDA, SVD, autoencoders etc.
Source: Wikipedia

This type of machine learning can be utilized for various facets of industries. For example, a marketing team could uncover valuable
insights about their customers; a group a customers could share similar attributes.

In this analysis, movie reviews are inspected to generate topics.

In [372]:
# import modules and libraries
# print out the versions of libraries, python version, and environment

import warnings
init_notebook_mode(connected=True)
warnings.filterwarnings('ignore')

import sys; print("Python", sys.version)
import pandas as pd; print("pandas", pd.__version__)
import numpy as np; print("numpy", np.__version__)
import sklearn; print("sklearn", sklearn.__version__)
import requests; print("requests", requests.__version__)
import nltk; print("nltk", nltk.__version__)
import re; print('re', re.__version__)
from requests import get; print("requests", requests.__version__)
from urllib.parse import urljoin; print("urllib.parse", urllib.request.__version__)
from bs4 import BeautifulSoup; print("bs4", bs4.__version__)
import nltk.stem.snowball; print("nltk.stem.snowball", nltk.__version__)
import spacy; print("spacy", spacy.__version__)
import pyLDAvis.sklearn; print("pyLDAvis", spacy.__version__)
import plotly; print("plotly", plotly.__version__)
from nltk.stem.snowball import SnowballStemmer
from sklearn.cluster import KMeans
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import os
import urllib
import bs4
import string
from collections import Counter
from sklearn.cluster import KMeans

Python 3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 14:01:38) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
pandas 0.20.3
numpy 1.15.4
sklearn 0.20.0
requests 2.14.2
nltk 3.3
re 2.2.1
requests 2.14.2
urllib.parse 3.6
bs4 4.6.3
nltk.stem.snowball 3.3
spacy 2.0.17
pyLDAvis 2.0.17
plotly 3.4.2


# Review Data

In [373]:
hw7data = pd.read_csv('/Users/MatthewPrzybyla/Downloads/hw7reviewdata.csv',encoding="latin-1")

In [374]:
reviewdata = hw7data.drop(['Unnamed: 0','type','label', 'file'],axis=1)
reviewdata.columns = ["review"]
reviewdata.head()

,review
0,Once again Mr. Costner has dragged out a movie...
1,This is an example of why the majority of acti...
2,"First of all I hate those moronic rappers, who..."
3,Not even the Beatles could write songs everyon...
4,Brass pictures (movies is not a fitting word f...


In [375]:
moviewReviews = reviewdata[:1000]

In [376]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [377]:
stemmer = SnowballStemmer("english")

In [378]:
totalvocab_stemmed = []
totalvocab_tokenized = []

for i in moviewReviews['review']:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'review', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [379]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 228823 items in vocab_frame


In [389]:
def tokens(x):
    return x.split(',')

In [390]:
tfidf_vect= TfidfVectorizer( tokenizer=tokens ,use_idf=True, smooth_idf=True, sublinear_tf=False)

In [391]:
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(moviewReviews['review']) #fit the vectorizer to synopses

In [392]:
terms = tfidf_vectorizer.get_feature_names()

In [393]:
dataframe = pd.DataFrame(moviewReviews['review'])

In [394]:
dataframe['review'].astype(str)
dataframe.head()

,review
0,Once again Mr. Costner has dragged out a movie...
1,This is an example of why the majority of acti...
2,"First of all I hate those moronic rappers, who..."
3,Not even the Beatles could write songs everyon...
4,Brass pictures (movies is not a fitting word f...


In [395]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)
print("Number of Stop Words wrt to spaCy is: ", len(stopwords))

Number of Stop Words wrt to spaCy is:  305


In [396]:
dataframe['review'] = dataframe['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)
                                                                   and word not in (punctuations)]))

In [397]:
dataframe.head()

,review
0,Once Mr. Costner dragged movie far longer nece...
1,This example majority action films same. Gener...
2,"First I hate moronic rappers, could'nt act gun..."
3,"Not Beatles write songs liked, Walter Hill mop..."
4,Brass pictures (movies fitting word them) some...


# Start of Final Project Analysis

# Hypothesis 

The hypothesis is:

Ho : LDA = NMF = LSI Models.

Ha : LDA ≠ NMF ≠ LSI Models.

The null hypothesis is that all models are the same. The alternative hypothesis is that all models are different.

In [398]:
# Creating a vectorizer
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(dataframe["review"])

In [399]:
NUM_TOPICS = 10

In [400]:
%%time

# Latent Dirichlet Allocation Model
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10
CPU times: user 4.66 s, sys: 47.6 ms, total: 4.71 s
Wall time: 4.76 s


In [401]:
%%time

# Non-Negative Matrix Factorization Model
nmf = NMF(n_components=NUM_TOPICS)
data_nmf = nmf.fit_transform(data_vectorized)

CPU times: user 785 ms, sys: 286 ms, total: 1.07 s
Wall time: 402 ms


In [402]:
%%time

# Latent Semantic Indexing Model using Truncated SVD
lsi = TruncatedSVD(n_components=NUM_TOPICS)
data_lsi = lsi.fit_transform(data_vectorized)

CPU times: user 50.5 ms, sys: 49.9 ms, total: 100 ms
Wall time: 28.1 ms


In [403]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

# Results A

The below results are for the LDA, NMF and LSI Models.

Shown are the top 10 topics of the models.

In [404]:
# Keywords for topics clustered by Latent Dirichlet Allocation
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('film', 316.6333877293193), ('characters', 130.26002853659293), ('story', 117.01883583481029), ('like', 113.23038089420484), ('movie', 101.68122410135862), ('scene', 84.29458305935508), ('character', 77.75584830793295), ('original', 66.58630359650287), ('films', 63.31735638507456), ('plot', 61.59394318392143)]
Topic 1:
[('belushi', 20.512740897375025), ('john', 20.29519094279515), ('randy', 19.393197244815898), ('wayne', 14.556713636237834), ('woodward', 11.354662578855933), ('sally', 10.531874892091), ('gang', 7.660474701480918), ('rides', 4.9706535118789015), ('westerns', 4.385460706067212), ('rogers', 4.181141166141913)]
Topic 2:
[('bishop', 45.62290364079268), ('larry', 30.871996001096058), ('hopper', 21.907662347813705), ('squad', 21.713045398612298), ('mod', 20.338072358796452), ('prison', 17.71837820254181), ('dennis', 17.541729053696777), ('ribisi', 17.090029779072218), ('danes', 17.064820444333193), ('tarantino', 16.63005990583501)]
Topic 3:
[('movie', 18

In [405]:
# Keywords for topics clustered by Latent Semantic Indexing
print("NMF Model:")
selected_topics(nmf, vectorizer)

NMF Model:
Topic 0:
[('movie', 11.772141033160667), ('watch', 0.6337236337093755), ('good', 0.6260703939568093), ('movies', 0.5379189333072966), ('know', 0.5026188660509258), ('sandra', 0.4528432887439634), ('people', 0.38748370488888173), ('seen', 0.38267118818077905), ('plot', 0.3615352527976532), ('think', 0.34381770150318447)]
Topic 1:
[('film', 7.908390190787888), ('films', 0.9313872939136315), ('time', 0.44510572136533044), ('vampire', 0.40379738335814175), ('director', 0.38570706933865506), ('fact', 0.2984809711856956), ('seen', 0.2950347288977789), ('feel', 0.2744653574802214), ('acting', 0.26258986771175563), ('think', 0.23160157676406895)]
Topic 2:
[('like', 7.420028479824756), ('guy', 1.0192567307783096), ('time', 1.010709534951721), ('look', 0.8880565605681037), ('people', 0.878533065430672), ('scene', 0.7388777741804591), ('way', 0.7070121784954931), ('prison', 0.6879990210115016), ('stupid', 0.5699815984858847), ('thing', 0.5664755308034349)]
Topic 3:
[('bone', 3.93096779

In [406]:
# Keywords for topics clustered by Non-Negative Matrix Factorization
print("LSI Model:")
selected_topics(lsi, vectorizer)

LSI Model:
Topic 0:
[('movie', 0.6836831252720709), ('film', 0.3261742781421991), ('like', 0.2403776806432977), ('bad', 0.16116795788107138), ('good', 0.13473480762157128), ('don', 0.11103373803501852), ('people', 0.11089690751168849), ('time', 0.10295162050797195), ('story', 0.0960784218661852), ('movies', 0.08910685893954524)]
Topic 1:
[('movie', 0.6016493998621232), ('sandra', 0.035476616163900486), ('bell', 0.013359596074746084), ('bullock', 0.012970223243172239), ('sick', 0.011845231500824439), ('himesh', 0.011145598012606697), ('spartans', 0.009649040651411938), ('scary', 0.009405125126497193), ('drake', 0.009350414289206252), ('stop', 0.00863902207017968)]
Topic 2:
[('like', 0.4327338251732372), ('people', 0.14516013937403333), ('don', 0.12018733450516578), ('way', 0.11550859759731993), ('story', 0.1088372724196813), ('guy', 0.10722501525737184), ('time', 0.09451004463104283), ('bone', 0.09339086973517999), ('eater', 0.08687372264167902), ('bad', 0.08516920113070442)]
Topic 3:
[

# Visual Summary of Results A (LDA)

Use this graph by hovering over a topic number to display the top terms.

These circles will become red and their respective top terms will show on the bar graph to the right.

The larger topics are more frequent and the closer the topics, the more the similar they are.

The selection of keywords is based on their frequency and discriminancy.

In [407]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=            Freq  cluster  topics          x           y
topic                                                   
5      53.053078        1       1   1.223200  101.310562
0      19.901750        1       2 -40.437012  -54.219624
9      17.532878        1       3  31.276657   52.846489
3       5.476453        1       4  -6.573143    0.908732
4       1.496628        1       5 -61.847416   96.902824
2       0.869616        1       6 -97.320305   44.567055
1       0.572554        1       7  58.106045    2.524526
8       0.554039        1       8 -32.642273   46.181622
7       0.393865        1       9  22.633875  -49.811535
6       0.149140        1      10 -70.493202   -5.756412, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
1799  Default  2062.000000         movie  2062.000000  30.0000  30.0000
1595  Default   892.000000          like   892.000000  29.0000  29.0000
3068  Default    90.000000         women    90.000000  28.0000  28.0000
1966  Default   383.000000        people   383.000000  27.0000  27.0000
1341  Default   154.000000        horror   154.000000  26.0000  26.0000
1627  Default   199.000000          look   199.000000  25.0000  25.0000
1636  Default   159.000000           lot   159.000000  24.0000  24.0000
204   Default   608.000000           bad   608.000000  23.0000  23.0000
1723  Default    89.000000           men    89.000000  22.0000  22.0000
1472  Default    71.000000          john    71.000000  21.0000  21.0000
2083  Default   158.000000        pretty   158.000000  20.0000  20.0000
688   Default    97.000000          dead    97.000000  19.0000  19.0000
260   Default   129.000000           big   129.000000  18.0000  18.0000
1800  Default   325.000000        movies   325.000000  17.0000  17.0000
440   Default   244.000000     character   244.000000  16.0000  16.0000
809   Default   440.000000           don   440.000000  15.0000  15.0000
1180  Default   505.000000          good   505.000000  14.0000  14.0000
754   Default   206.000000          didn   206.000000  13.0000  13.0000
2423  Default   127.000000        series   127.000000  12.0000  12.0000
615   Default    79.000000        couple    79.000000  11.0000  11.0000
1232  Default    83.000000          half    83.000000  10.0000  10.0000
1058  Default   221.000000         films   221.000000   9.0000   9.0000
2675  Default   132.000000        stupid   132.000000   8.0000   8.0000
255   Default   148.000000          best   148.000000   7.0000   7.0000
598   Default    60.000000          cool    60.000000   6.0000   6.0000
2880  Default   107.000000        trying   107.000000   5.0000   5.0000
271   Default    40.000000        bishop    40.000000   4.0000   4.0000
1509  Default    84.000000        killer    84.000000   3.0000   3.0000
1523  Default   276.000000          know   276.000000   2.0000   2.0000
257   Default   263.000000        better   263.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
2545  Topic10     0.034818        sooner     5.042822   1.5325  -8.0741
600   Topic10     0.034465        copies     4.995075   1.5318  -8.0843
2992  Topic10     0.034409          wake     4.993276   1.5305  -8.0859
1420  Topic10     0.034625  intellectual     5.025168   1.5304  -8.0797
576   Topic10     0.034524       contain     5.024963   1.5275  -8.0826
252   Topic10     0.034681         belly     5.123987   1.5125  -8.0781
2585  Topic10     0.034585      sporting     5.067428   1.5209  -8.0808
3008  Topic10     0.034916     warehouse     5.364575   1.4734  -8.0713
1082  Topic10     0.034769        flying     5.258781   1.4891  -8.0755
1845  Topic10     0.034869        no-one     5.751886   1.4023  -8.0727
1577  Topic10     0.040276           let    88.982617  -1.1924  -7.9285
183   Topic10     0.036149    australian    11.948639   0.7073  -8.0366
470   Topic10     0.035021      c

NMF is shown below.

In [408]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(nmf, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=            Freq  cluster  topics          x          y
topic                                                  
1      17.922363        1       1 -68.786354  54.521423
0      17.623591        1       2 -44.956371  27.697660
6      14.046317        1       3 -90.272461  -4.933486
2      13.168177        1       4 -33.918198  58.222267
8      11.660660        1       5 -54.804436   0.451046
5       9.547792        1       6 -19.251888   3.001414
9       5.548600        1       7 -80.511246  25.149256
7       4.332326        1       8 -65.835182 -30.072769
4       3.834128        1       9  -9.480170  33.079140
3       2.316046        1      10 -30.967749 -26.372314, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
1799  Default  2857.000000       movie  2857.000000  30.0000  30.0000
1052  Default  1716.000000        film  1716.000000  29.0000  29.0000
204   Default   705.000000         bad   705.000000  28.0000  28.0000
1595  Default   874.000000        like   874.000000  27.0000  27.0000
809   Default   370.000000         don   370.000000  26.0000  26.0000
1180  Default   483.000000        good   483.000000  25.0000  25.0000
2648  Default   303.000000       story   303.000000  24.0000  24.0000
1966  Default   387.000000      people   387.000000  23.0000  23.0000
300   Default    97.000000        bone    97.000000  22.0000  22.0000
863   Default    94.000000       eater    94.000000  21.0000  21.0000
2469  Default   105.000000        sick   105.000000  20.0000  20.0000
440   Default   242.000000   character   242.000000  19.0000  19.0000
955   Default   103.000000    exorcism   103.000000  18.0000  18.0000
1841  Default    91.000000      nicole    91.000000  17.0000  17.0000
1058  Default   284.000000       films   284.000000  16.0000  16.0000
770   Default   230.000000    director   230.000000  15.0000  15.0000
2260  Default   112.000000        rest   112.000000  14.0000  14.0000
2641  Default    91.000000        stop    91.000000  13.0000  13.0000
1509  Default    81.000000      killer    81.000000  12.0000  12.0000
1523  Default   302.000000        know   302.000000  11.0000  11.0000
3017  Default   338.000000       watch   338.000000  10.0000  10.0000
1341  Default   154.000000      horror   154.000000   9.0000   9.0000
3091  Default   160.000000       worst   160.000000   8.0000   8.0000
2950  Default    65.000000      valley    65.000000   7.0000   7.0000
277   Default    65.000000  blackwater    65.000000   6.0000   6.0000
2890  Default    58.000000     twisted    58.000000   5.0000   5.0000
2798  Default   266.000000       think   266.000000   4.0000   4.0000
2448  Default    62.000000     sheriff    62.000000   3.0000   3.0000
25    Default   305.000000      acting   305.000000   2.0000   2.0000
2406  Default   288.000000        seen   288.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
865   Topic10     2.359734        eats     4.702993   3.0757  -6.6007
580   Topic10     3.738803     contest     7.535388   3.0645  -6.1405
430   Topic10    11.826684         cgi    23.859000   3.0635  -4.9889
620   Topic10     3.793720      cowboy     7.814843   3.0426  -6.1259
738   Topic10     2.077708  destroying     4.297419   3.0386  -6.7280
867   Topic10     3.692190        edge     7.660331   3.0355  -6.1530
733   Topic10     3.760884   desperate     7.860840   3.0281  -6.1346
95    Topic10    10.977431     ancient    23.191616   3.0174  -5.0634
1818  Topic10    11.227749       names    24.683698   2.9776  -5.0408
199   Topic10     6.429892         axe    14.913728   2.9240  -5.5983
1543  Topic10     8.256729       large    22.445042   2.7653  -5.3482
2372  Topic10    16.296177      sci-fi    59.725505   2.4665  -4.6683
1468  Topic10    10.328872         jim    35.187343   2.5396  -5.1243
438   Topic10     8.194443     channel    25.047120   2.6480  -5.

Below we are visualizing the LSI(SVD) scatterplot.

The data for 2 topics is shown.

The similarity between keywords which is measured by distance with the markers is displayed by dots.

In [409]:
svd_2d = TruncatedSVD(n_components=2)
data_2d = svd_2d.fit_transform(data_vectorized)

In [410]:
py.sign_in('PythonAPI', 'ubpiol2cve')

In [411]:
trace = go.Scattergl(
    x = data_2d[:,0],
    y = data_2d[:,1],
    mode = 'markers',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    ),
    text = vectorizer.get_feature_names(),
    hovertext = vectorizer.get_feature_names(),
    hoverinfo = 'text' 
)
data = [trace]
iplot(data, filename='scatter-mode')

In this scatterplot the actual text is shown.

At first glance, this data looks messy but can actually be zoomed in for better visuals.

In [412]:
trace = go.Scattergl(
    x = data_2d[:,0],
    y = data_2d[:,1],
    mode = 'text',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    ),
    text = vectorizer.get_feature_names()
)
data = [trace]
iplot(data, filename='text-scatter-mode')

In [413]:
dataframe = dataframe[~dataframe['review'].str.contains("br", "br br")]

In [414]:
def spacy_bigram_tokenizer(phrase):
    doc = parser(phrase) # create spacy object
    token_not_noun = []
    notnoun_noun_list = []
    noun = ""

    for item in doc:
        if item.pos_ != "NOUN": # separate nouns and not nouns
            token_not_noun.append(item.text)
        if item.pos_ == "NOUN":
            noun = item.text
        
        for notnoun in token_not_noun:
            notnoun_noun_list.append(notnoun + " " + noun)

    return " ".join([i for i in notnoun_noun_list])

In [415]:
bivectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, ngram_range=(1,2))
bigram_vectorized = bivectorizer.fit_transform(dataframe["review"])

In [416]:
bi_lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_bi_lda = bi_lda.fit_transform(bigram_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


# Results B

Here we see the Bi-LDA model which using a bivectorization instead.

In [417]:
print("Bi-LDA Model:")
selected_topics(bi_lda, bivectorizer)

Bi-LDA Model:
Topic 0:
[('movie', 17.367738537025193), ('know', 5.354793127092447), ('expected', 4.4654253750348785), ('book', 4.070176927803961), ('subject', 3.6736629158145906), ('minutes', 3.155111627058195), ('fine', 3.0624037962840527), ('stupid', 2.87438309244078), ('sure', 2.647740249346526), ('family', 2.5690697861850005)]
Topic 1:
[('episode', 4.9877897350590334), ('pretty', 3.179420929085435), ('church', 2.2312864283995935), ('big', 1.8572460254079028), ('self', 1.5821857041672356), ('plays', 1.570240298634208), ('wants', 1.552518122973365), ('john', 1.53672272376347), ('type', 1.4406421595322576), ('like', 1.3235049005823782)]
Topic 2:
[('men', 9.817608972148527), ('women', 8.863726918984014), ('minutes', 6.808435853789321), ('male', 5.054186510234402), ('way', 4.14857576237979), ('like', 3.6518615444879603), ('given', 3.3409116083862016), ('little', 3.3198483651918034), ('shot', 3.2279493273932043), ('felt', 3.1083630391829744)]
Topic 3:
[('story', 20.601534683872906), ('st

# Visual Summary of Bi-LDA Model

Use this graph by hovering over a topic number to display the top terms.

These circles will become red and their respective top terms will show on the bar graph to the right.

The larger topics are more frequent and the closer the topics, the more the similar they are.

The selection of keywords is based on their frequency and discriminancy.

In [418]:
bi_dash = pyLDAvis.sklearn.prepare(bi_lda, bigram_vectorized, bivectorizer, mds='tsne')
bi_dash

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
9      82.972940        1       1  -94.591331  -65.314301
4       3.678125        1       2 -117.117386  182.412003
5       2.559427        1       3   30.995659  219.359848
7       2.336782        1       4  -26.395605   74.064148
3       2.224196        1       5  145.333633 -130.977020
0       2.111992        1       6   54.593197  -22.643948
2       1.829624        1       7   -2.780024 -167.931702
6       1.087089        1       8  122.781746  116.728119
1       0.892081        1       9 -179.499344   43.084789
8       0.307745        1      10  207.706253    8.350459, topic_info=     Category        Freq             Term       Total  loglift  logprob
term                                                                    
478   Default  522.000000            movie  522.000000  30.0000  30.0000
259   Default  268.000000             film  268.000000  29.0000  29.0000
413   Default  200.000000             like  200.000000  28.0000  28.0000
689   Default   75.000000            story   75.000000  27.0000  27.0000
59    Default  167.000000              bad  167.000000  26.0000  26.0000
731   Default  111.000000             time  111.000000  25.0000  25.0000
548   Default   84.000000             plot   84.000000  24.0000  24.0000
388   Default   58.000000             know   58.000000  23.0000  23.0000
724   Default   64.000000            thing   64.000000  22.0000  22.0000
807   Default   63.000000              way   63.000000  21.0000  21.0000
632   Default   78.000000             seen   78.000000  20.0000  20.0000
16    Default   83.000000           acting   83.000000  19.0000  19.0000
563   Default   31.000000           pretty   31.000000  18.0000  18.0000
801   Default   91.000000            watch   91.000000  17.0000  17.0000
469   Default   45.000000          minutes   45.000000  16.0000  16.0000
301   Default  104.000000             good  104.000000  15.0000  15.0000
74    Default   57.000000           better   57.000000  14.0000  14.0000
774   Default   53.000000               ve   53.000000  13.0000  13.0000
629   Default   40.000000           script   40.000000  12.0000  12.0000
111   Default   67.000000       characters   67.000000  11.0000  11.0000
421   Default   43.000000           little   43.000000  10.0000  10.0000
110   Default   43.000000        character   43.000000   9.0000   9.0000
410   Default   33.000000             life   33.000000   8.0000   8.0000
75    Default   34.000000              big   34.000000   7.0000   7.0000
804   Default   62.000000         watching   62.000000   6.0000   6.0000
417   Default   29.000000             line   29.000000   5.0000   5.0000
305   Default   47.000000              got   47.000000   4.0000   4.0000
532   Default   77.000000           people   77.000000   3.0000   3.0000
621   Default   57.000000            scene   57.000000   2.0000   2.0000
189   Default   39.000000            doesn   39.000000   1.0000   1.0000
...       ...         ...              ...         ...      ...      ...
586   Topic10    0.042204              red    4.415814   1.1332  -6.8410
565   Topic10    0.040721            prior    4.266691   1.1318  -6.8767
560   Topic10    0.051142              pre    5.359003   1.1317  -6.6489
432   Topic10    0.051073  looking forward    5.403309   1.1221  -6.6502
173   Topic10    0.082262           dialog    9.295187   1.0563  -6.1736
618   Topic10    0.087537             says   10.387962   1.0073  -6.1114
567   Topic10    0.052634          problem    5.762349   1.0879  -6.6201
342   Topic10    0.050904           hoping    5.535767   1.0946  -6.6535
663   Topic10    0.089765             soon   11.331258   0.9455  -6.0863
22    Topic10    0.055448           actual    6.225607   1.0627  -6.5680
794   Topic10    0.085745            wants   11.068077   0.9232  -6.1321
409   Topic10    0.056266          letting    6.5

# Conclusion

In the LDA model, the first two topics were the most apparent in terms of frequency. 
These terms included, but not limited to: movie, film, bad, good, people, know, and watch. 
In the second largest topic, terms included but were not limited to: film, movie, like, bad, acting, scenes, story, plot.
The first topic appears to be a cluster of terms describing, obviously the movie, but the audience as well. In the second topic,
The structure of the movie seemed to be the motif of the review (i.e., scenes, story, and plot).

In the NMF model, most of the top topics shared similar frequencies. The largest included terms like film, time, vampire, director, seen, feel,think, and look. Most of these terms seem to be describing what a person feels when they are watching a movie, or what they think the
creator of the movie wanted the audience to feel (i.e., seen, feel, think, and look). Once clear outlier in terms of cohesive pattern in the
topic is vampire. This situation is where NMF seems different.

In the LSI model, a different, graphical approach was taken due to the non-negative natrix factorization.
It appears that. The first topic covers general terms like bad, good, story, and movie, while the second topic is different in that is has the names of people in the movie like drake, sandra, and bullock; also includes negative words like sick, scary, and stop, suggesting horror as a genre, or a negative feeling toward the movie.

The Bi-LDA model has the biggest seperation in topic size. It resembles LDA in its first topic, while the second topic seems to have some
contradicting terms like worst and comedy. It is interesting to note that a majority of the top topics are incredibly small in topic size, 
such as terms like watching, waste, unfortunately, stinks, and leave, summarizing a negative review.

Final Thoughts

The alternative hypothesis is supported

Overall, with all of the approaches considered, it seems that NMF is the best in terms of producing an accurate, balanced, and interesting
topic spread. At first, I thought LDA was a better methodology, as it has proven to be considered more like a machine learning algorithm with more robust power and meaning, but when reviewing the use case, I would personally recommend using the NMF
model because it could produce the most summaries, and would not have to worry about having topics with little importance in terms of
frequency. I would rather have more big bubbles, than two big bubbles where the rest are small in regards to the visualizations. I think
the reviews could be summed better with various topics occuring frequently. I also thought it was notable that the NMF model picked up on
reviews that talked almost directly to sentiment like 'feel' and 'think'. These are strong words to use as a reviewer. I think both models can be useful in different ways, depending on the business goal.

For future works, I would like to implement these different models to my work as a Business Analyst to gain insights on customer behavior,
as well as key topics from phone conversations in optimization and aquisition campaigns.